In [2]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env

In [3]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = '''
SELECT id, name, start_date, end_date, program_id, date, time, path, user_id, cohort_id, ip, created_at, updated_at
From cohorts as c 
join logs as l on c.id = l.cohort_id;
'''
df = pd.read_sql(query, url)
df.head()

,id,name,start_date,end_date,program_id,date,time,path,user_id,cohort_id,ip,created_at,updated_at
0,8,Hampton,2015-09-22,2016-02-06,1,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,2016-06-14 19:52:26,2016-06-14 19:52:26
1,8,Hampton,2015-09-22,2016-02-06,1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,2016-06-14 19:52:26,2016-06-14 19:52:26
2,8,Hampton,2015-09-22,2016-02-06,1,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,2016-06-14 19:52:26,2016-06-14 19:52:26
3,8,Hampton,2015-09-22,2016-02-06,1,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,2016-06-14 19:52:26,2016-06-14 19:52:26
4,22,Teddy,2018-01-08,2018-05-17,2,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018-01-08 13:59:10,2018-01-08 13:59:10


### Label students by the program they are in.

In [4]:
df.path.value_counts()

/                                                               45854
javascript-i                                                    18203
toc                                                             17591
search/search_index.json                                        17534
java-iii                                                        13166
                                                                ...  
content/examples/javascript/primitive-types.html                    1
content/examples/javascript/conditionals.html                       1
2-storytelling/1-overview/www.qlik.com                              1
syntax-types-and-variables                                          1
appendix/professional-development/post-interview-review-form        1
Name: path, Length: 2224, dtype: int64

In [6]:
df.name.value_counts()

Staff         84031
Ceres         40730
Zion          38096
Jupiter       37109
Fortuna       36902
Voyageurs     35636
Ganymede      33844
Apex          33568
Deimos        32888
Darden        32015
Teddy         30926
Hyperion      29855
Betelgeuse    29356
Ulysses       28534
Europa        28033
Xanadu        27749
Bayes         26538
Wrangell      25586
Andromeda     25359
Kalypso       23691
Curie         21582
Yosemite      20743
Bash          17713
Luna          16623
Marco         16397
Easley        14715
Lassen         9587
Arches         8890
Florence       8562
Sequoia        7444
Neptune        7276
Olympic        4954
Kings          2845
Pinnacles      2158
Hampton        1712
Oberon         1672
Quincy         1237
Niagara         755
Mammoth         691
Glacier         598
Joshua          302
Ike             253
Badlands         93
Franklin         72
Apollo            5
Denali            4
Everglades        1
Name: name, dtype: int64

### Is it possible to identify the user_id of a staff member?

In [ ]:
# Ste date column as datetime dtype and then sset as index 
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)

In [ ]:
df.info()


In [ ]:
pages = df['path'].resample('d').count()
pages.head()

In [ ]:
pages.plot()

### Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

In [ ]:
df["month"] = df.index.month_name()

In [ ]:
df.head()

In [ ]:
df.start_date = pd.to_datetime(df.start_date)

In [ ]:
df.head()

In [ ]:
df.end_date = pd.to_datetime(df.end_date)

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(df.start_date, df.end_date)

In [ ]:
def one_user_df_prep(df, user):
    '''
    This function returns a dataframe consisting of data for only a single defined user
    '''
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages_one_user = df['path'].resample('d').count()
    return pages_one_user

In [ ]:
def compute_pct_b(pages_one_user, span, weight, user):
    '''
    This function adds the %b of a bollinger band range for the page views of a single user's log activity
    '''
    # Calculate upper and lower bollinger band
    midband = pages_one_user.ewm(span=span).mean()
    stdev = pages_one_user.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    
    # Add upper and lower band values to dataframe
    bb = pd.concat([ub, lb], axis=1)
    
    # Combine all data into a single dataframe
    my_df = pd.concat([pages_one_user, midband, bb], axis=1)
    my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
    
    # Calculate percent b and relevant user id to dataframe
    my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

In [ ]:
def plot_bands(my_df, user):
    '''
    This function plots the bolliger bands of the page views for a single user
    '''
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages_one_user, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

In [ ]:
def find_anomalies(df, user, span, weight, plot=False):
    '''
    This function returns the records where a user's daily activity exceeded the upper limit of a bollinger band range
    '''
    
    # Reduce dataframe to represent a single user
    pages_one_user = one_user_df_prep(df, user)
    
    # Add bollinger band data to dataframe
    my_df = compute_pct_b(pages_one_user, span, weight, user)
    
    # Plot data if requested (plot=True)
    if plot:
        plot_bands(my_df, user)
    
    # Return only records that sit outside of bollinger band upper limit
    return my_df[my_df.pct_b>1]

### Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

In [ ]:
user = 1
span = 30
weight = 6
user_df = find_anomalies(df, user, span, weight)

anomalies = pd.DataFrame()
user_df = find_anomalies(df, user, span, weight)
anomalies = pd.concat([anomalies, user_df], axis=0)

In [ ]:
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [ ]:
anomalies

In [ ]:
anomalies.sort_values(by='pages_one_user', ascending=False)

In [ ]:
df_341 = one_user_df_prep(df, 341)

In [ ]:
df_341.plot(figsize=(16,8))

In [ ]:
df_146 = one_user_df_prep(df, 146)

In [ ]:
df_146.plot(figsize=(16,8))

In [ ]:
anomalies.user_id.value_counts()

In [ ]:
df.iloc[146]

In [ ]:
df_80 = one_user_df_prep(df, 80)

#I think this one looks weird 

In [ ]:
df_80.plot(figsize=(16,8))

In [ ]:
df.iloc[80]

In [ ]:
df_18 = one_user_df_prep(df, 18)
df_18.plot(figsize=(16,8))

#this kinda looks normal??

In [ ]:
df.iloc[18]

### Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2.

In [ ]:
span = 30

# compute midband
midband = pages.ewm(span=span).mean()

midband.head()

In [ ]:
stdev = pages.ewm(span=span).std()

stdev.head()


In [ ]:
ub = midband + stdev*2
lb = midband - stdev*2
ub, lb

In [ ]:
# concatenate ub and lb together into one df, bb
bb = pd.concat([ub, lb], axis=1)

bb.columns = ['ub', 'lb']
bb.head()

### Plot the access activity of these students.